# Amazon Chronos

In [2]:
from chronos import BaseChronosPipeline
import torch
import pandas as pd
import numpy as np
from methodology.forecasting import compute_rmse_df

/Users/henry/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the data and calculate the inflation rate
cpi_monthly_data = pd.read_csv("methodology/data/cpi_monthly_data.csv", index_col=0)
cpi = cpi_monthly_data[["00"]]
inflation_rate = cpi.pct_change(12).dropna(how="all") * 100
inflation_rate.columns = ["Inflation Rate"]
inflation_rate.index = pd.to_datetime(inflation_rate.index)

In [4]:
dates = inflation_rate.loc["2009-12-01":"2024-11-01"].index
models = ["Chronos (Tiny)", "Chronos (Base)", "Chronos (Large)"]
chronos_df = pd.DataFrame(index=dates, columns=pd.MultiIndex.from_product([models, range(1, 13)]))

In [5]:
# Chronos (Tiny)
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny", 
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

for date in dates:
    # Get the inflation rate data up to the current time step
    past_data = inflation_rate.loc[:date].to_numpy().flatten()
    # Make a forecast for the next 12 months
    quantiles, mean = pipeline.predict_quantiles(
        context=torch.tensor(past_data),
        prediction_length=12,
        quantile_levels=[0.1, 0.5, 0.9],
    )
    chronos_df.loc[date, "Chronos (Tiny)"] = mean.flatten().numpy()

In [9]:
# Chronos (Base)
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base", 
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

for date in dates:
    # Get the inflation rate data up to the current time step
    past_data = inflation_rate.loc[:date].to_numpy().flatten()
    # Make a forecast for the next 12 months
    quantiles, mean = pipeline.predict_quantiles(
        context=torch.tensor(past_data),
        prediction_length=12,
        quantile_levels=[0.1, 0.5, 0.9],
    )
    chronos_df.loc[date, "Chronos (Base)"] = mean.flatten().numpy()

In [13]:
# Chronos (Large)
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large", 
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

for date in dates:
    # Get the inflation rate data up to the current time step
    past_data = inflation_rate.loc[:date].to_numpy().flatten()
    # Make a forecast for the next 12 months
    quantiles, mean = pipeline.predict_quantiles(
        context=torch.tensor(past_data),
        prediction_length=12,
        quantile_levels=[0.1, 0.5, 0.9],
    )
    chronos_df.loc[date, "Chronos (Large)"] = mean.flatten().numpy()

In [14]:
compute_rmse_df(chronos_df)

,Chronos (Tiny),Chronos (Base),Chronos (Large)
1,0.414401,0.432217,0.429231
2,0.632304,0.655870,0.647211
3,0.835744,0.839278,0.831425
4,1.053941,1.059065,1.041964
5,1.291431,1.288409,1.269807
6,1.531713,1.497005,1.473674
7,1.764678,1.723224,1.688165
8,2.001192,1.949767,1.900940
9,2.221752,2.154053,2.096347
10,2.416971,2.320989,2.261194


In [15]:
chronos_df.to_csv("results/benchmarks/chronos_forecasts.csv")